In [1]:
import os
import sys
import glob

import pyproj
import cmocean
import netCDF4
import PIL.Image
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

from plotting import tile

EPSG3857 = pyproj.Proj('EPSG:3857')
TILE3857 = tile.Tile3857()

In [2]:
data = os.path.join(os.getenv('DATA'), 'LiveOcean_output/f2019.11.06')
files = sorted(glob.glob(f'{data}/*.nc'))
target = 'tmp'
if not os.path.exists(target):
    os.mkdir(target)
variables = iter(['temp', 'zeta', 'u', 'v'])

In [3]:
nc = netCDF4.MFDataset(f'{data}/*.nc')
varname = next(variables)
zoom = 10

In [8]:
# Extract spatial variables
lon = nc.variables['lon_rho'][:]
lat = nc.variables['lat_rho'][:]
msk = nc.variables['mask_rho'][:]
lo,la = EPSG3857(lon,lat) # project EPSG:4326 to EPSG:3857 (web mercator)

# temporal
time = nc.variables['ocean_time']
time = netCDF4.num2date(time[:], units=time.units)

# data
d = nc.variables[varname]
d = d[:]

# image size based on tiles
mla = la[msk == 0]
mlo = lo[msk == 0]
lrt = TILE3857.tile(mlo.max(), mla.min(), zoom)
lrb = TILE3857.bounds(*lrt)
ult = TILE3857.tile(mlo.min(), mla.max(), zoom)
ulb = TILE3857.bounds(*ult)
ntx = lrt[0] - ult[0] + 1 # number of tiles in x
nty = lrt[1] - ult[1] + 1 # number of tiles in y

In [6]:
# loop times in file
for t in range(len(time)):

    d = d[t, :] # select time

    # check for vertical coordinate
    if d.ndim > 2:
        d = d[-1,:] # surface is last in ROMS

    # apply mask
    d = np.ma.masked_where(msk == 0, d)

    # pcolor uses surrounding points, if any are masked, mask this cell
    #   see https://matplotlib.org/api/_as_gen/matplotlib.pyplot.pcolor.html
    d[:-1,:] = np.ma.masked_where(msk[1:,:] == 0, d[:-1,:])
    d[:,:-1] = np.ma.masked_where(msk[:,1:] == 0, d[:,:-1])

    # image size/resolution
    #dpi = 256
    dpi = 96   # suitable for screen and web
    height = nty * dpi
    width  = ntx * dpi

    fig = plt.figure(dpi=dpi, facecolor='none', edgecolor='none')
    fig.set_alpha(0)
    fig.set_figheight(height/dpi)
    fig.set_figwidth(width/dpi)

    ax = fig.add_axes([0., 0., 1., 1.], xticks=[], yticks=[])
    ax.set_axis_off()

    # pcolor
    pcolor = ax.pcolor(lo, la, d, cmap=plt.get_cmap('viridis'), edgecolors='none', linewidth=0.05)

    ax.set_frame_on(False)
    ax.set_clip_on(False)
    ax.set_position([0, 0, 1, 1])

    # set limits based on tiles
    ax.set_xlim(ulb[0], lrb[2])
    ax.set_ylim(lrb[1], ulb[3])

    # File output
    #prefix = ncfile.split('/')[-1][:-3]
    prefix = 'liveocean'
    filename = f'{target}/{prefix}_{varname}.png'
    fig.savefig(filename, dpi=dpi, bbox_inches='tight', pad_inches=0.0, transparent=True)

    plt.close(fig)

(663,)